In [1]:
import tensorflow as tf
import tensorflow_datasets as tfds

## Downloading and preprocessing the data

In [2]:
BUFFER_SIZE = 70_000
BATCH_SIZE = 128
NUM_EPOCHS = 20

In [3]:
mnist_dataset, mnist_info = tfds.load(name = 'mnist', with_info=True, as_supervised=True)

In [4]:
mnist_train, mnist_test = mnist_dataset['train'], mnist_dataset['test']

In [5]:
def scale(image, label):
    image = tf.cast(image, tf.float32)
    image /= 255.0

    return image, label

In [6]:
train_and_validation_data = mnist_train.map(scale)
test_data = mnist_test.map(scale)

In [7]:
num_validation_samples = 0.1 * mnist_info.splits['train'].num_examples
num_validation_samples = tf.cast(num_validation_samples, tf.int64)

In [8]:
num_test_samples = mnist_info.splits['test'].num_examples
num_test_samples = tf.cast(num_test_samples, tf.int64)

In [9]:
train_and_validation_data = train_and_validation_data.shuffle(BUFFER_SIZE)

In [10]:
train_data = train_and_validation_data.skip(num_validation_samples)

In [11]:
validation_data = train_and_validation_data.take(num_validation_samples)

In [12]:
train_data = train_data.batch(BATCH_SIZE)
validation_data = validation_data.batch(num_validation_samples)
test_data = test_data.batch(num_test_samples)

## Creating the model and training it
- Define the architecture of the model
- Compile the model
- Train the model

In [13]:
model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(50, 5, activation = 'relu', input_shape = (28,28,1)),
    tf.keras.layers.MaxPooling2D(pool_size = (2,2)),
    tf.keras.layers.Conv2D(50, 3, activation = 'relu'),
    tf.keras.layers.MaxPooling2D(pool_size = (2,2)),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(10)
])

c:\Users\SANDY\anaconda3\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [14]:
model.summary(line_length=75)

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━┓
┃ Layer (type)                   ┃ Output Shape            ┃      Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                │ (None, 24, 24, 50)      │        1,300 │
├────────────────────────────────┼─────────────────────────┼──────────────┤
│ max_pooling2d (MaxPooling2D)   │ (None, 12, 12, 50)      │            0 │
├────────────────────────────────┼─────────────────────────┼──────────────┤
│ conv2d_1 (Conv2D)              │ (None, 10, 10, 50)      │       22,550 │
├────────────────────────────────┼─────────────────────────┼──────────────┤
│ max_pooling2d_1 (MaxPooling2D) │ (None, 5, 5, 50)        │            0 │
├────────────────────────────────┼─────────────────────────┼──────────────┤
│ flatten (Flatten)              │ (None, 1250)            │            0 │
├────────────────────────────────┼─────────────────────────┼──────────────┤
│ dense (Dense)                  │ (None, 10)              │       12,510 │
└────────────────────────────────┴─────────────────────────┴──────────────┘

 Total params: 36,360 (142.03 KB)

 Trainable params: 36,360 (142.03 KB)

 Non-trainable params: 0 (0.00 B)

In [15]:
loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

In [16]:
model.compile(optimizer='adam', loss = loss_fn, metrics=['accuracy'])

In [17]:
early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor = 'val_loss',
    mode = 'auto',
    min_delta = 0,
    patience = 2,
    verbose = 0,
    restore_best_weights = True
)

In [18]:
model.fit(
    train_data,
    epochs = NUM_EPOCHS,
    callbacks = [early_stopping],
    validation_data = validation_data,
    verbose = 2
)

Epoch 1/20
422/422 - 14s - 34ms/step - accuracy: 0.9277 - loss: 0.2603 - val_accuracy: 0.9718 - val_loss: 0.0900
Epoch 2/20
422/422 - 14s - 32ms/step - accuracy: 0.9793 - loss: 0.0692 - val_accuracy: 0.9853 - val_loss: 0.0571
Epoch 3/20
422/422 - 12s - 29ms/step - accuracy: 0.9841 - loss: 0.0509 - val_accuracy: 0.9838 - val_loss: 0.0488
Epoch 4/20
422/422 - 16s - 38ms/step - accuracy: 0.9877 - loss: 0.0408 - val_accuracy: 0.9912 - val_loss: 0.0250
Epoch 5/20
422/422 - 19s - 45ms/step - accuracy: 0.9883 - loss: 0.0368 - val_accuracy: 0.9913 - val_loss: 0.0323
Epoch 6/20
422/422 - 37s - 88ms/step - accuracy: 0.9903 - loss: 0.0313 - val_accuracy: 0.9918 - val_loss: 0.0224
Epoch 7/20
422/422 - 20s - 48ms/step - accuracy: 0.9914 - loss: 0.0277 - val_accuracy: 0.9907 - val_loss: 0.0307
Epoch 8/20
422/422 - 18s - 42ms/step - accuracy: 0.9925 - loss: 0.0245 - val_accuracy: 0.9947 - val_loss: 0.0195
Epoch 9/20
422/422 - 19s - 45ms/step - accuracy: 0.9932 - loss: 0.0212 - val_accuracy: 0.9958 - 